<a href="https://colab.research.google.com/github/douglaswcp/Jogo-do-Sabido/blob/main/Dados_Sabido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from IPython.display import clear_output, display, HTML
import ipywidgets as widgets

# Definição dos dados
dados = {
    "Vermelho": [2, 20, 11, 24, 8, 16],
    "Azul": [3, 18, 10, 21, 5, 13],
    "Amarelo": [1, 6, 12, 14, 17, 22],
    "Verde": [4, 9, 7, 15, 19, 7]
}

# Cores em HTML
cores_html = {
    "Vermelho": "#ff4c4c",
    "Azul": "#4c6aff",
    "Amarelo": "#ffd633",
    "Verde": "#33cc33"
}

# Lista para armazenar resultados já usados
resultados_usados = set()
rodada_atual = {"num": 0}

def renderizar_dados(jogada, titulo=""):
    """Mostra os dados como quadrados coloridos em HTML"""
    html = f"<h3 style='margin-bottom:10px;'>{titulo}</h3><div style='display:flex; gap:10px;'>"
    for cor, valor in jogada:
        html += f"""
        <div style='
            width:70px; height:70px;
            background:{cores_html[cor]};
            display:flex; align-items:center; justify-content:center;
            border-radius:12px;
            font-size:20px; font-weight:bold;
            color:black;
            box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
        '>{valor}</div>
        """
    html += "</div>"
    display(HTML(html))

def girar_dados_instantaneo():
    """Gira os 4 dados e mostra o resultado final instantaneamente"""
    rodada_atual["num"] += 1

    # Garantir jogada inédita
    for _ in range(1000):
        jogada = [(cor, random.choice(valores)) for cor, valores in dados.items()]
        jogada_ordenada = sorted(jogada, key=lambda x: x[1], reverse=True)
        jogada_tupla = tuple(jogada_ordenada)

        if jogada_tupla not in resultados_usados:
            resultados_usados.add(jogada_tupla)
            break
    else:
        clear_output(wait=True)
        display(HTML("<h3>⚠️ Todas as combinações possíveis já foram usadas!</h3>"))
        return None

    # Mostrar resultado final
    clear_output(wait=True)
    renderizar_dados(jogada_ordenada, f"✅ Rodada {rodada_atual['num']} - Resultado")

    # Exibe o botão novamente para a próxima rodada
    display(botao)

# Criar botão
botao = widgets.Button(description="🎲 Girar Dados", button_style='success')

def ao_clicar(b):
    clear_output(wait=True)  # Limpa a saída anterior antes do novo giro
    girar_dados_instantaneo()

botao.on_click(ao_clicar)

# Exibe o botão na primeira vez que o código é executado
display(botao)

Button(button_style='success', description='🎲 Girar Dados', style=ButtonStyle())

In [ ]:
import random
from IPython.display import clear_output, display, HTML
import ipywidgets as widgets
import json, uuid
from IPython.display import HTML, display

# --- SUBSTITUIÇÃO: animação 3D maior e de frente ---
import json, uuid
from IPython.display import HTML, display

def animar_dados_3d(jogada, rodada_id):
    """
    Cena 3D com 4 dados coloridos, girando e parando com o número sorteado
    na face frontal (voltada para a câmera). Visualização maior e de frente.
    """
    scene_id = f"scene_{rodada_id}_{uuid.uuid4().hex[:6]}"

    html = f"""
    <div id="{scene_id}" style="
         width:100%;
         max-width: 1200px;
         height: 520px;
         border-radius:12px;
         background:#111;
         margin:12px 0;
         position:relative;
         overflow:hidden;
    "></div>
    <script>
    (function() {{
      const CONTAINER_ID = "{scene_id}";
      const jogada = {json.dumps(jogada, ensure_ascii=False)};
      const dadosMap = {json.dumps(dados, ensure_ascii=False)};
      const coresMap = {json.dumps(cores_html, ensure_ascii=False)};

      function loadScript(src) {{
        return new Promise((resolve, reject) => {{
          if (window.__threejs_loaded) return resolve();
          const s = document.createElement('script');
          s.src = src;
          s.onload = () => {{ window.__threejs_loaded = true; resolve(); }};
          s.onerror = reject;
          document.body.appendChild(s);
        }});
      }}

      loadScript("https://unpkg.com/three@0.152.2/build/three.min.js").then(() => {{
        const container = document.getElementById(CONTAINER_ID);
        if (!container) return;

        // Renderer
        const renderer = new THREE.WebGLRenderer({{ antialias:true, alpha:true }});
        const w = container.clientWidth, h = container.clientHeight;
        renderer.setSize(w, h);
        renderer.setPixelRatio(Math.min(window.devicePixelRatio, 2));
        container.appendChild(renderer.domElement);

        // Cena e câmera (frontal)
        const scene = new THREE.Scene();
        const camera = new THREE.PerspectiveCamera(28, w/h, 0.1, 100);
        camera.position.set(0, 0, 9.5); // frontal, sem inclinação
        camera.lookAt(0, 0, 0);
        scene.add(camera);

        // Luzes
        scene.add(new THREE.AmbientLight(0xffffff, 0.9));
        const dir = new THREE.DirectionalLight(0xffffff, 0.9);
        dir.position.set(3,5,6);
        scene.add(dir);

        // Piso opcional (ajuda na noção de espaço)
        const floorGeo = new THREE.PlaneGeometry(50,50);
        const floorMat = new THREE.MeshStandardMaterial({{ color: 0x222222, roughness:0.9, metalness:0.0 }});
        const floor = new THREE.Mesh(floorGeo, floorMat);
        floor.rotation.x = -Math.PI/2;
        floor.position.y = -1.2;
        floor.receiveShadow = true;
        scene.add(floor);

        // Gera material com número
        function makeFaceTexture(color, text) {{
          const size = 256;
          const canvas = document.createElement('canvas');
          canvas.width = canvas.height = size;
          const ctx = canvas.getContext('2d');

          // fundo
          ctx.fillStyle = color;
          ctx.fillRect(0,0,size,size);

          // borda
          ctx.strokeStyle = '#00000055';
          ctx.lineWidth = 8;
          ctx.strokeRect(4,4,size-8,size-8);

          // número
          ctx.fillStyle = '#111';
          ctx.font = 'bold 120px Inter, Arial, sans-serif';
          ctx.textAlign = 'center';
          ctx.textBaseline = 'middle';
          ctx.fillText(String(text), size/2, size/2);

          const tex = new THREE.CanvasTexture(canvas);
          tex.needsUpdate = true;
          return new THREE.MeshStandardMaterial({{ map: tex }});
        }}

        // Cria um dado com 6 faces (ordem THREE.BoxGeometry):
        // [right(+X), left(-X), top(+Y), bottom(-Y), front(+Z), back(-Z)]
        function makeDie(cor, valores) {{
          const mats = valores.map(v => makeFaceTexture(coresMap[cor], v));
          while (mats.length < 6) mats.push(mats[0]);
          const geom = new THREE.BoxGeometry(1.6, 1.6, 1.6); // ligeiramente maior
          const mesh = new THREE.Mesh(geom, mats);
          mesh.castShadow = true;
          mesh.receiveShadow = true;
          return mesh;
        }}

        // Distribui 4 dados
        const dice = [];
        const spacing = 2.4;
        const startX = -((jogada.length - 1) * spacing)/2;

        jogada.forEach((par, i) => {{
          const [cor, valor] = par;
          const die = makeDie(cor, dadosMap[cor]);
          die.position.set(startX + i*spacing, 0, 0);
          die.rotation.set(Math.random()*Math.PI, Math.random()*Math.PI, Math.random()*Math.PI);
          scene.add(die);
          dice.push({{
            mesh: die,
            cor: cor,
            alvo: valor,
            vel: new THREE.Vector3(
              (Math.random()*2+1)*(Math.random()<0.5?-1:1),
              (Math.random()*2+1)*(Math.random()<0.5?-1:1),
              (Math.random()*2+1)*(Math.random()<0.5?-1:1)
            )
          }});
        }});

        const clock = new THREE.Clock();
        let elapsed = 0;
        const rollDuration = 1.4;
        const settleDuration = 0.5;
        let phase = 'roll';

        // Agora o número sorteado fica na face FRONTAL (+Z, índice 4)
        function setFrontFaceToValue(mesh, cor, valor) {{
          const frontMat = makeFaceTexture(coresMap[cor], valor);
          const mats = Array.isArray(mesh.material) ? mesh.material.slice() : [mesh.material];
          mats[4] = frontMat;
          mesh.material = mats;
        }}

        // Ajuda a "zerar" rotações para ficar totalmente de frente
        function easeToFrontRotation(mesh, ease) {{
          // reduz rotações X/Z a zero e aproxima Y de 0 (frente para +Z)
          mesh.rotation.x *= (1 - ease);
          mesh.rotation.z *= (1 - ease);
          // traga Y a 0 suavemente
          mesh.rotation.y *= (1 - ease);
          // estabiliza Y muito próximo de 0
          if (Math.abs(mesh.rotation.y) < 0.001) mesh.rotation.y = 0;
        }}

        function animate() {{
          const dt = clock.getDelta();
          elapsed += dt;

          if (phase === 'roll') {{
            dice.forEach(d => {{
              d.mesh.rotation.x += d.vel.x * dt;
              d.mesh.rotation.y += d.vel.y * dt;
              d.mesh.rotation.z += d.vel.z * dt;
              // leve “quique” vertical
              d.mesh.position.y = Math.sin((elapsed + d.mesh.position.x)*4) * 0.15;
            }});
            if (elapsed >= rollDuration) {{
              dice.forEach(d => setFrontFaceToValue(d.mesh, d.cor, d.alvo));
              phase = 'settle';
              elapsed = 0;
            }}
          }} else if (phase === 'settle') {{
            const t = Math.min(elapsed/settleDuration, 1);
            const ease = 1 - Math.pow(1 - t, 3); // ease-out
            dice.forEach(d => {{
              easeToFrontRotation(d.mesh, ease);
              d.mesh.position.y *= (1 - ease);
            }});
            if (t >= 1) {{
              // Garante frente perfeita
              dice.forEach(d => {{
                d.mesh.rotation.set(0, 0, 0);
                d.mesh.position.y = 0;
              }});
              phase = 'stopped';
            }}
          }}

          renderer.render(scene, camera);
          if (phase !== 'stopped') requestAnimationFrame(animate);
        }}

        animate();

        // Responsivo
        const ro = new ResizeObserver(entries => {{
          for (const entry of entries) {{
            const cr = entry.contentRect;
            renderer.setSize(cr.width, cr.height);
            camera.aspect = cr.width / cr.height;
            camera.updateProjectionMatrix();
          }}
        }});
        ro.observe(container);

      }}).catch(err => {{
        const container = document.getElementById(CONTAINER_ID);
        if (container) container.innerHTML = '<div style="color:#fff;padding:12px;">Falha ao carregar a animação 3D.</div>';
        console.error(err);
      }});
    }})();
    </script>
    """
    display(HTML(html))
# --- FIM DA ADIÇÃO ---


# Definição dos dados
dados = {
    "Vermelho": [2, 20, 11, 24, 8, 16],
    "Azul": [3, 18, 10, 21, 5, 13],
    "Amarelo": [1, 6, 12, 14, 17, 22],
    "Verde": [4, 9, 7, 15, 19, 7]
}

# Cores em HTML
cores_html = {
    "Vermelho": "#ff4c4c",
    "Azul": "#4c6aff",
    "Amarelo": "#ffd633",
    "Verde": "#33cc33"
}

# Lista para armazenar resultados já usados
resultados_usados = set()
rodada_atual = {"num": 0}

def renderizar_dados(jogada, titulo=""):
    """Mostra os dados como quadrados coloridos em HTML"""
    html = f"<h3 style='margin-bottom:10px;'>{titulo}</h3><div style='display:flex; gap:10px;'>"
    for cor, valor in jogada:
        html += f"""
        <div style='
            width:70px; height:70px;
            background:{cores_html[cor]};
            display:flex; align-items:center; justify-content:center;
            border-radius:12px;
            font-size:20px; font-weight:bold;
            color:black;
            box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
        '>{valor}</div>
        """
    html += "</div>"
    display(HTML(html))

def girar_dados_instantaneo():
    """Gira os 4 dados, mostra o resultado final e anima em 3D"""
    rodada_atual["num"] += 1

    # Garantir jogada inédita
    for _ in range(1000):
        jogada = [(cor, random.choice(valores)) for cor, valores in dados.items()]
        jogada_ordenada = sorted(jogada, key=lambda x: x[1], reverse=True)
        jogada_tupla = tuple(jogada_ordenada)

        if jogada_tupla not in resultados_usados:
            resultados_usados.add(jogada_tupla)
            break
    else:
        clear_output(wait=True)
        display(HTML("<h3>⚠️ Todas as combinações possíveis já foram usadas!</h3>"))
        return None

    # Mostrar animação 3D + resultado final
    clear_output(wait=True)

    # 🎬 Animação 3D (novo)
    animar_dados_3d(jogada_ordenada, rodada_atual["num"])

    # ✅ Render antigo (mantido)
    renderizar_dados(jogada_ordenada, f"✅ Rodada {rodada_atual['num']} - Resultado")

    # Exibe o botão novamente para a próxima rodada
    display(botao)

# Criar botão
botao = widgets.Button(description="🎲 Girar Dados", button_style='success')

def ao_clicar(b):
    clear_output(wait=True)  # Limpa a saída anterior antes do novo giro
    girar_dados_instantaneo()

botao.on_click(ao_clicar)

# Exibe o botão na primeira vez que o código é executado
display(botao)

Button(button_style='success', description='🎲 Girar Dados', style=ButtonStyle())